Extract course duration and Who should take it

In [1]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# -------------------- DRIVER SETUP --------------------
def get_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# -------------------- CLEANING FUNCTION --------------------
def clean_text(text):
    if not text:
        return None
    text = re.sub(r"<.*?>", "", text)  # remove HTML tags
    text = re.sub(r"\s+", " ", text)   # normalize spaces
    return text.strip()

# -------------------- WHO SHOULD ATTEND EXTRACTION --------------------
def extract_who_should_attend(soup):
    # Step 1: look for <strong> or <span> containing 'Who Should Attend the Course?'
    candidate = soup.find(['strong','span'], string=re.compile(r"Who Should Attend the Course\?", re.I))
    
    if candidate:
        ul = candidate.find_next('ul')
        if ul:
            who_list = [li.get_text(" ", strip=True) for li in ul.find_all('li') if li.get_text(strip=True)]
            if who_list:
                return "\n".join(who_list)

    # Step 2: fallback to general patterns
    patterns = [
        r"Who Should Attend",
        r"Who Should Enroll",
        r"Who can Enroll",
        r"Target Audience",
        r"Ideal Candidates?",
        r"Who Can Join",
        r"Eligibility",
        r"Who is this (course|program) for"
    ]

    who_should_attend = []
    headers = soup.find_all(['h1','h2','h3','h4','h5','h6','strong'])
    for header in headers:
        header_text = header.get_text(strip=True)
        if any(re.search(pattern, header_text, re.I) for pattern in patterns):
            ul = header.find_next('ul')
            if ul:
                for li in ul.find_all('li'):
                    text = li.get_text(" ", strip=True)
                    if text:
                        who_should_attend.append(text)
            break

    if who_should_attend:
        seen = set()
        unique_items = []
        for item in who_should_attend:
            normalized = item.lower().strip()
            if normalized not in seen:
                seen.add(normalized)
                unique_items.append(item.strip())
        return "\n".join(unique_items)

    return "N/A - Section not found"

# -------------------- DURATION EXTRACTION --------------------
def extract_duration_info(soup):
    duration_info = []
    
    duration_divs = soup.find_all('div', class_=re.compile(r'shadow.*box-border.*relative.*mt-2.*rounded-md.*bg-white.*py-2.*px-2'))
    
    for div in duration_divs:
        h5_element = div.find('h5', class_=re.compile(r'text-\[16px\].*font-medium.*text-purple'))
        if h5_element:
            duration_text = h5_element.get_text(strip=True)
            if any(keyword in duration_text.lower() for keyword in ['hour','session','week','month','duration']):
                duration_info.append(duration_text)
    
    if not duration_info:
        h5_elements = soup.find_all('h5')
        for h5 in h5_elements:
            h5_text = h5.get_text(strip=True)
            if any(keyword in h5_text.lower() for keyword in ['hour','session','week','month','duration']):
                duration_info.append(h5_text)
    
    if not duration_info:
        all_elements = soup.find_all(string=re.compile(r'\d+\s*(Hours?|Hrs|Sessions?|Weeks?|Months?)', re.I))
        for element in all_elements:
            if hasattr(element, 'parent'):
                parent = element.parent
                if parent.name not in ['script','style']:
                    text = parent.get_text(strip=True)
                    if re.search(r'\d+\s*(Hours?|Hrs|Sessions?|Weeks?|Months?)', text, re.I):
                        duration_info.append(text)
    
    if duration_info:
        seen = set()
        unique_duration = []
        for item in duration_info:
            if item not in seen:
                seen.add(item)
                unique_duration.append(item)
        return " | ".join(unique_duration[:3])
    else:
        return "N/A - Duration not found"

# -------------------- SCRAPER FUNCTION --------------------
def scrape_course_data(course_url):
    driver = get_driver(headless=False)
    driver.get(course_url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    try:
        title_tag = soup.find("h1")
        course_name = clean_text(title_tag.get_text()) if title_tag else None

        who_should_attend = extract_who_should_attend(soup)
        duration_info = extract_duration_info(soup)

        return {
            "Course Name": course_name,
            "Course Link": course_url,
            "Who Should Attend": who_should_attend,
            "Duration": duration_info
        }

    except Exception as e:
        return {
            "Course Name": None,
            "Course Link": course_url,
            "Who Should Attend": "N/A",
            "Duration": "N/A",
            "Error": str(e)
        }

# -------------------- MAIN --------------------
urls = [
    "https://www.henryharvin.com/car-designing-course"
]

all_data = []
for url in urls:
    print(f"\nScraping: {url}")
    course_data = scrape_course_data(url)
    print(f"Course Name: {course_data['Course Name']}")
    print(f"Duration: {course_data['Duration']}")
    print("Who Should Attend:\n" + course_data['Who Should Attend'])
    print(f"Course Link: {course_data['Course Link']}")
    all_data.append(course_data)

output_file = r"C:\Users\taslim.siddiqui\Downloads\HenryHarvin_Personality_Development.xlsx"
df = pd.DataFrame(all_data)
df.to_excel(output_file, index=False)
print(f"\n✅ Data saved to {output_file}")



Scraping: https://www.henryharvin.com/car-designing-course
Course Name: Car Designing Course
Duration: 48 Hours of Instructor-Led Sessions | 16 Hours of Live Interactive Doubt Solving Sessions | 8 Hours of Live Master Sessions by Industry Experts
Who Should Attend:
Aspiring Car Designer
Fresh Graduates
Design Intern
Vehicle Artists
Product Designer
Course Link: https://www.henryharvin.com/car-designing-course

✅ Data saved to C:\Users\taslim.siddiqui\Downloads\HenryHarvin_Personality_Development.xlsx


In [4]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# -------------------- DRIVER SETUP --------------------
def get_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# -------------------- CLEANING FUNCTION --------------------
def clean_text(text):
    if not text:
        return None
    text = re.sub(r"<.*?>", "", text)  # remove HTML tags
    text = re.sub(r"\s+", " ", text)   # normalize spaces
    return text.strip()


# -------------------- WHO SHOULD ATTEND EXTRACTION --------------------
def extract_who_should_attend(soup):
    # Step 1: look for <strong> or <span> containing 'Who Should Attend the Course?'
    candidate = soup.find(['strong','span'], string=re.compile(r"Who Should Attend the Course\?", re.I))
    
    if candidate:
        ul = candidate.find_next('ul')
        if ul:
            who_list = [li.get_text(" ", strip=True) for li in ul.find_all('li') if li.get_text(strip=True)]
            if who_list:
                return "\n".join(who_list)

    # Step 2: fallback to general patterns
    patterns = [
        r"Who Should Attend",
        r"Who Should Enroll",
        r"Who can Enroll",
        r"Target Audience",
        r"Ideal Candidates?",
        r"Who Can Join",
        r"Eligibility",
        r"Who is this (course|program) for"
    ]

    who_should_attend = []
    headers = soup.find_all(['h1','h2','h3','h4','h5','h6','strong'])
    for header in headers:
        header_text = header.get_text(strip=True)
        if any(re.search(pattern, header_text, re.I) for pattern in patterns):
            ul = header.find_next('ul')
            if ul:
                for li in ul.find_all('li'):
                    text = li.get_text(" ", strip=True)
                    if text:
                        who_should_attend.append(text)
            break

    if who_should_attend:
        seen = set()
        unique_items = []
        for item in who_should_attend:
            normalized = item.lower().strip()
            if normalized not in seen:
                seen.add(normalized)
                unique_items.append(item.strip())
        return "\n".join(unique_items)

    return "N/A - Section not found"

# -------------------- DURATION EXTRACTION --------------------
def extract_duration_info(soup):
    duration_info = []
    
    duration_divs = soup.find_all('div', class_=re.compile(r'shadow.*box-border.*relative.*mt-2.*rounded-md.*bg-white.*py-2.*px-2'))
    
    for div in duration_divs:
        h5_element = div.find('h5', class_=re.compile(r'text-\[16px\].*font-medium.*text-purple'))
        if h5_element:
            duration_text = h5_element.get_text(strip=True)
            if any(keyword in duration_text.lower() for keyword in ['hour','session','week','month','duration']):
                duration_info.append(duration_text)
    
    if not duration_info:
        h5_elements = soup.find_all('h5')
        for h5 in h5_elements:
            h5_text = h5.get_text(strip=True)
            if any(keyword in h5_text.lower() for keyword in ['hour','session','week','month','duration']):
                duration_info.append(h5_text)
    
    if not duration_info:
        all_elements = soup.find_all(string=re.compile(r'\d+\s*(Hours?|Hrs|Sessions?|Weeks?|Months?)', re.I))
        for element in all_elements:
            if hasattr(element, 'parent'):
                parent = element.parent
                if parent.name not in ['script','style']:
                    text = parent.get_text(strip=True)
                    if re.search(r'\d+\s*(Hours?|Hrs|Sessions?|Weeks?|Months?)', text, re.I):
                        duration_info.append(text)
    
    if duration_info:
        seen = set()
        unique_duration = []
        for item in duration_info:
            if item not in seen:
                seen.add(item)
                unique_duration.append(item)
        return " | ".join(unique_duration[:3])
    else:
        return "N/A - Duration not found"

# -------------------- SCRAPER FUNCTION --------------------
def scrape_course_data(course_url):
    driver = get_driver(headless=False)
    driver.get(course_url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    try:
        title_tag = soup.find("h1")
        course_name = clean_text(title_tag.get_text()) if title_tag else None
        who_should_attend = extract_who_should_attend(soup)
        duration_info = extract_duration_info(soup)

        return {
            "Course Name": course_name,
            "Course Link": course_url,
            "Who Should Attend": who_should_attend,
            "Duration": duration_info
        }
    except Exception as e:
        return {
            "Course Name": None,
            "Course Link": course_url,
            "Who Should Attend": "N/A",
            "Duration": "N/A",
            "Error": str(e)
        }

# -------------------- MAIN --------------------
# 📥 Read input Excel file (must contain column "Course Link")
input_file = r"C:\Users\taslim.siddiqui\Downloads\Henry_Harvin_course_link.xlsx"
df_input = pd.read_excel(input_file)

all_data = []
for url in df_input["Course Link"].dropna():
    print(f"\nScraping: {url}")
    course_data = scrape_course_data(url)
    print(f"Course Name: {course_data['Course Name']}")
    print(f"Duration: {course_data['Duration']}")
    print(f"Who Should Attend: {course_data['Who Should Attend']}")
    print(f"Course Link: {course_data['Course Link']}")
    all_data.append(course_data)

# 📤 Save results to output Excel
output_file = r"C:\Users\taslim.siddiqui\Downloads\HH_Duration and who should tak.xlsx"
df_out = pd.DataFrame(all_data)
df_out.to_excel(output_file, index=False)
print(f"\n✅ Data saved to {output_file}")



Scraping: https://www.henryharvin.com/iso-31000-lead-risk-manager-training-course
Course Name: ISO 31000 Lead Risk Manager Training Course
Duration: 40 Hours of Instructor-Led Sessions | 13 Hours of Live Interactive Doubt Solving Sessions | 7 Hours of Live Master Sessions by Industry Experts
Who Should Attend: IT Professionals
Risk Specialists
Business Managers
Risk Consultants
Project Management Personnel
IT Architects
Risk Advisors
Risk Directors
Upper-level management seeking risk management knowledge to integrate it into strategic initiatives
Individuals responsible for creating Risk Management processes in an organization
Individuals who are responsible for the creation and protection of value in organizations
Professionals looking to gain a comprehensive understanding of Risk Management Concepts
Individuals who are looking to sit for PECBs ISO 31000 Lead Risk Manager Certification exam
Course Link: https://www.henryharvin.com/iso-31000-lead-risk-manager-training-course

Scraping

KeyboardInterrupt: 

fast Extract Price

In [ ]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import os

# -------------------- DRIVER SETUP --------------------
def get_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# -------------------- PRICE EXTRACTION FUNCTIONS --------------------
def extract_price_from_html(html_content):
    """
    Extract price from HTML content using BeautifulSoup
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Method 1: Find span containing ₹ symbol
    price_span = soup.find('span', string=re.compile(r'₹\s*\d+'))
    if price_span:
        price_text = price_span.get_text(strip=True)
        return extract_numeric_price(price_text)
    
    # Method 2: Find any span with price-like content
    spans = soup.find_all('span')
    for span in spans:
        text = span.get_text(strip=True)
        if '₹' in text and any(char.isdigit() for char in text):
            return extract_numeric_price(text)
    
    # Method 3: Look for price in other elements
    price_elements = soup.find_all(string=re.compile(r'₹\s*\d+'))
    for element in price_elements:
        price_text = element.strip()
        price = extract_numeric_price(price_text)
        if price:
            return price
    
    return None

def extract_numeric_price(price_text):
    """
    Extract numeric value from price text
    """
    # Remove ₹ symbol and any non-digit characters except decimal point
    price_clean = re.sub(r'[^\d.]', '', price_text.replace('₹', '').strip())
    
    try:
        return float(price_clean) if price_clean else None
    except ValueError:
        return None

def extract_course_price(driver, url):
    """
    Extract course price from the given URL
    """
    try:
        print(f"🌐 Navigating to: {url}")
        driver.get(url)
        
        # Wait for the page to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        # Wait a bit more for dynamic content
        time.sleep(4)
        
        # Try multiple selectors for price
        price_selectors = [
            "span",  # Direct span
            "td span",  # Span inside td
            ".text-[14px] span",  # Based on your class
            "[class*='price']",  # Any element with price in class
            "[class*='amount']",  # Any element with amount in class
            "[class*='cost']",  # Any element with cost in class
            "[class*='fee']",  # Any element with fee in class
            "div span",  # Span inside div
            "b, strong",  # Bold or strong tags
            ".price, .amount, .cost, .fee",  # Common price classes
            "ins span",  # Span inside ins tag (common for discounted prices)
            "del span",  # Span inside del tag (common for original prices)
        ]
        
        for selector in price_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    text = element.text.strip()
                    if text and '₹' in text and any(char.isdigit() for char in text):
                        price = extract_numeric_price(text)
                        if price:
                            print(f"✅ Price found with selector '{selector}': ₹{price}")
                            return price
            except Exception as e:
                continue
        
        # If no price found with selectors, try getting page source and parsing with BeautifulSoup
        page_source = driver.page_source
        price = extract_price_from_html(page_source)
        if price:
            print(f"✅ Price found via page source: ₹{price}")
            return price
        
        print("❌ No price found on the page")
        return None
        
    except Exception as e:
        print(f"❌ Error extracting price from {url}: {str(e)}")
        return None

# -------------------- EXCEL PROCESSING FUNCTIONS --------------------
def read_urls_from_excel(file_path, url_column="URL"):
    """
    Read URLs from Excel file
    """
    try:
        # Try different engines and sheet names
        try:
            df = pd.read_excel(file_path, engine='openpyxl')
        except:
            try:
                df = pd.read_excel(file_path, engine='xlrd')
            except:
                df = pd.read_excel(file_path)
        
        # Auto-detect URL column if not specified
        if url_column not in df.columns:
            url_candidates = [col for col in df.columns if 'url' in col.lower() or 'link' in col.lower()]
            if url_candidates:
                url_column = url_candidates[0]
                print(f"🔍 Auto-detected URL column: {url_column}")
            else:
                url_column = df.columns[0]  # Use first column
                print(f"⚠️  Using first column as URL column: {url_column}")
        
        urls = df[url_column].dropna().astype(str).tolist()
        print(f"📖 Read {len(urls)} URLs from {file_path}")
        return urls, df
        
    except Exception as e:
        print(f"❌ Error reading Excel file: {str(e)}")
        return [], None

def save_results_to_excel(original_df, results, output_file):
    """
    Save results to Excel file
    """
    try:
        # Create results DataFrame
        results_df = pd.DataFrame(results)
        
        # Merge with original data if available
        if original_df is not None:
            # Add original data
            final_df = original_df.copy()
            final_df['Extracted_Price'] = None
            final_df['Extraction_Status'] = None
            final_df['Extraction_Timestamp'] = None
            
            # Update with results
            for result in results:
                url = result['URL']
                price = result['Price']
                status = result['Status']
                timestamp = result['Timestamp']
                
                # Find matching row in original DataFrame
                mask = final_df.iloc[:, 0].astype(str) == url
                if mask.any():
                    idx = mask.idxmax()
                    final_df.loc[idx, 'Extracted_Price'] = price
                    final_df.loc[idx, 'Extraction_Status'] = status
                    final_df.loc[idx, 'Extraction_Timestamp'] = timestamp
        else:
            final_df = results_df
        
        # Save to Excel
        final_df.to_excel(output_file, index=False, engine='openpyxl')
        print(f"💾 Results saved to: {output_file}")
        return final_df
        
    except Exception as e:
        print(f"❌ Error saving to Excel: {str(e)}")
        return None

# -------------------- MAIN PROCESSING FUNCTION --------------------
def process_excel_urls(input_file, output_file=None, url_column="URL", headless=True):
    """
    Main function to process URLs from Excel file and extract prices
    """
    if output_file is None:
        base_name = os.path.splitext(input_file)[0]
        output_file = f"{base_name}_with_prices.xlsx"
    
    print(f"🚀 Starting price extraction process")
    print(f"📥 Input file: {input_file}")
    print(f"📤 Output file: {output_file}")
    
    # Read URLs from Excel
    urls, original_df = read_urls_from_excel(input_file, url_column)
    
    if not urls:
        print("❌ No URLs found to process")
        return False
    
    driver = None
    results = []
    
    try:
        driver = get_driver(headless=headless)
        
        for i, url in enumerate(urls, 1):
            print(f"\n{'='*60}")
            print(f"📊 Processing {i}/{len(urls)}: {url}")
            
            # Clean URL
            url = url.strip()
            if not url.startswith('http'):
                url = 'https://' + url
            
            # Extract price
            start_time = time.time()
            price = extract_course_price(driver, url)
            extraction_time = time.time() - start_time
            
            # Determine status
            status = "Success" if price else "Failed"
            
            # Prepare result
            result = {
                'URL': url,
                'Price': price if price else "Not Found",
                'Status': status,
                'Timestamp': pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
                'Extraction_Time_Seconds': round(extraction_time, 2)
            }
            
            results.append(result)
            
            print(f"✅ Completed: {i}/{len(urls)} | Status: {status} | Price: ₹{price if price else 'N/A'}")
            
            # Wait between requests to be respectful
            if i < len(urls):
                time.sleep(2)
        
        # Save results
        final_df = save_results_to_excel(original_df, results, output_file)
        
        # Print summary
        print(f"\n{'='*60}")
        print("🎯 EXTRACTION SUMMARY")
        print(f"{'='*60}")
        successful = sum(1 for r in results if r['Status'] == 'Success')
        failed = len(results) - successful
        print(f"✅ Successful: {successful}")
        print(f"❌ Failed: {failed}")
        print(f"📊 Total: {len(results)}")
        
        if successful > 0:
            prices = [r['Price'] for r in results if isinstance(r['Price'], (int, float))]
            if prices:
                print(f"💰 Min Price: ₹{min(prices)}")
                print(f"💰 Max Price: ₹{max(prices)}")
                print(f"💰 Average Price: ₹{sum(prices)/len(prices):.2f}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error in main processing: {str(e)}")
        return False
        
    finally:
        if driver:
            driver.quit()
            print("\n🔚 WebDriver closed")

# -------------------- USAGE EXAMPLES --------------------
if __name__ == "__main__":
    # Example 1: Basic usage with default settings
    input_excel = r"C:\Users\taslim.siddiqui\Downloads\Henry H price Extraction 2.xlsx"  # Change this to your Excel file path
    output_excel = r"C:\Users\taslim.siddiqui\Downloads\course_urls_with_prices-1.xlsx"
    
    # Example 2: Custom column name (if your URL column has a different name)
    # process_excel_urls(input_excel, output_excel, url_column="Course_URL")
    
    # Example 3: Non-headless mode (to see browser)
    # process_excel_urls(input_excel, output_excel, headless=False)
    
    # Run the main function
    success = process_excel_urls(input_excel, output_excel)
    
    if success:
        print(f"\n🎉 Price extraction completed successfully!")
        print(f"📁 Check the output file: {output_excel}")
    else:
        print(f"\n💥 Price extraction failed!")